# YOLO Trainning Code
### Prerequisite Step
**This portion of our code deals with loading the Google Drive on to the local directory of Google Collab**

In [1]:
# Enable GPU in google collab if not on this portion will flag error.
!nvidia-smi

Sun Jul 31 21:46:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 Machine_learning_course	   RecommenderSystem.ipynb
 Machine_Learning.ipynb		   RecommendorSystem
'My Drive'			   yolov3
'Plant Disease Detection System'   YoloV3_Trainning.ipynb


### 1) Clone the Darknet
**This Porition is essentially responsible for cloning Darknet because Yolo was developed using Darknet framework and runs only on that particular framework**



In [2]:
!git clone https://github.com/AlexeyAB/darknet\

Cloning into 'darknet'...
remote: Enumerating objects: 15460, done.
remote: Total 15460 (delta 0), reused 0 (delta 0), pack-reused 15460
Receiving objects: 100% (15460/15460), 14.07 MiB | 5.60 MiB/s, done.
Resolving deltas: 100% (10385/10385), done.


### 2) Compile Darknet using Nvidia GPU
**This porition is responsible for Compiling Darknet using Nvidia GPU and making slight adjustments to make file**



In [3]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

### 3) Configure Darknet network for training YOLO V3

In [4]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [5]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [6]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [7]:
!echo "Glasses" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/MyDrive/yolov3' > data/obj.data
!mkdir data/obj

In [8]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-08-01 00:26:15--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   108MB/s    in 1.4s    

2022-08-01 00:26:16 (108 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



### 4) Extract Images

**The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" if any other name used .zip file and folder name is adjustment accordingly on Google Drive**

In [9]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
   creating: data/obj/images/
  inflating: data/obj/images/0000000015411.jpg  
  inflating: data/obj/images/0000000015411.txt  
  inflating: data/obj/images/0002_ORBIT_Glasses_Front_Deep_Etch_600x.jpg  
  inflating: data/obj/images/0002_ORBIT_Glasses_Front_Deep_Etch_600x.txt  
  inflating: data/obj/images/087776froyal00000144_1.jpg  
  inflating: data/obj/images/087776froyal00000144_1.txt  
  inflating: data/obj/images/10033_f_2_1.jpg  
  inflating: data/obj/images/10033_f_2_1.txt  
  inflating: data/obj/images/150564527-109126091-1597476903.jpg  
  inflating: data/obj/images/150564527-109126091-1597476903.txt  
  inflating: data/obj/images/1634059661-zenni-optical-clear-glasses-1634059648.jpg  
  inflating: data/obj/images/1634059661-zenni-optical-clear-glasses-1634059648.txt  
  inflating: data/obj/images/1_276_2.jpg  
  inflating: data/obj/images/1_276_2.txt  
  inflating: data/obj/images/21044892-front-940x529.jpg  
  inflating: data/obj/images/

In [10]:
#This part is entirely optional
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.515817 0.459578 0.929701 0.465729

['0', '0.515817', '0.459578', '0.929701', '0.465729']
0 data/obj/images/61fdQ9DAeCL._UX569_.txt
0 0.515817 0.459578 0.929701 0.465729
0 0.494444 0.461111 0.900000 0.425926

['0', '0.494444', '0.461111', '0.900000', '0.425926']
1 data/obj/images/3112_1233_6110_050_270x270.txt
0 0.494444 0.461111 0.900000 0.425926
0 0.491667 0.520833 0.910000 0.660714

['0', '0.491667', '0.520833', '0.910000', '0.660714']
2 data/obj/images/images96.txt
0 0.491667 0.520833 0.910000 0.660714
15 0.486702 0.506616 0.815957 0.574669

['15', '0.486702', '0.506616', '0.815957', '0.574669']
3 data/obj/images/21044892-front-940x529.txt
0 0.486702 0.506616 0.815957 0.574669
0 0.503333 0.645833 0.933333 0.708333

['0', '0.503333', '0.645833', '0.933333', '0.708333']
4 data/obj/images/images87.txt
0 0.503333 0.645833 0.933333 0.708333
0 0.512000 0.546000 0.928000 0.456000

['0', '0.512000', '0.546000', '0.928000', '0.456000']
5 data/obj/images/metal-optical-frames-500x500.txt
0

In [11]:
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/images148.jpg', 'data/obj/images/0002_ORBIT_Glasses_Front_Deep_Etch_600x.jpg', 'data/obj/images/images94.jpg', 'data/obj/images/21044892-front-940x529.jpg', 'data/obj/images/61FdewVBjML._UX569_.jpg', 'data/obj/images/3112_1233_6110_050_270x270.jpg', 'data/obj/images/images106.jpg', 'data/obj/images/71eGvavJlML._AC_UX385_.jpg', 'data/obj/images/images105.jpg', 'data/obj/images/images147.jpg', 'data/obj/images/5-BRUTA2-47O-GR_2.jpg', 'data/obj/images/10033_f_2_1.jpg', 'data/obj/images/71IPuW2MPDL._AC_SY355_.jpg', 'data/obj/images/images102.jpg', 'data/obj/images/photo-of-black-female-spectacle-frames-BXA4J6.jpg', 'data/obj/images/images144.jpg', 'data/obj/images/Spectacles-Business-Frames-Optical-Eyeglasses-Men.jpg', 'data/obj/images/1634059661-zenni-optical-clear-glasses-1634059648.jpg', 'data/obj/images/metal-optical-frames-500x500.jpg', 'data/obj/images/087776froyal00000144_1.jpg', 'data/obj/images/images86.jpg', 'data/obj/images/150564527-109126091-1597476903.jpg', 

In [12]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

### 6) Start the training

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 124525, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.885267), count: 4, class_loss = 0.000311, iou_loss = 0.032861, total_loss = 0.033171 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 124529, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.910852), count: 4, class_loss = 0.000004, iou_loss = 0.031908, total_loss = 0.031912 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (ms